In [1]:
import torch
from torch import tensor


We now illustrate implementing matrix multiplication using torch operations on torch tensors. First, we use an ad-hoc implementation of the dot product.

In [2]:
def matmul(a, b):
    (ar, ac), (br, bc) = a.shape, b.shape
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc):
            c[i, j] = (a[i, :] * b[:, j]).sum()
    return c


We setup our test data.

In [3]:
from pathlib import Path
from urllib.request import urlretrieve
import gzip, pickle

MNIST_URL='https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/data/mnist.pkl.gz?raw=true'
path_data = Path('data')
path_data.mkdir(exist_ok=True)
path_gz = path_data/'mnist.pkl.gz'

if not path_gz.exists():
    urlretrieve(MNIST_URL, path_gz)

with gzip.open(path_gz, 'rb') as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')

x_train,y_train,x_valid,y_valid = map(tensor, (x_train,y_train,x_valid,y_valid))

torch.manual_seed(1)
weights = torch.randn(784, 10)
bias = torch.zeros(10)

m1 = x_valid[:5]
m2 = weights
ar, ac = m1.shape 
br, bc = m2.shape

t1 = torch.zeros(ar, bc)

for i in range(ar):         # 5
    for j in range(bc):     # 10
        for k in range(ac): # 784
            t1[i, j] += m1[i, k] * m2[k, j]


Verify correctness.

In [4]:
from fastcore.all import *

test_close(t1, matmul(m1, m2))


We see that torch on CPU with the ad-hoc dot product is slightly slower than `njit`.

In [5]:
%timeit -n 50 _ = matmul(m1, m2)


859 µs ± 100 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)


We now consider using PyTorch's dot product subroutine.

In [6]:
def matmul(a, b):
    (ar, ac), (br, bc) = a.shape, b.shape
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc):
            c[i, j] = torch.dot(a[i, :], b[:, j])
    return c


In [7]:
test_close(t1, matmul(m1, m2))


PyTorch tensors on CPU with builtin dot product is still slightly slower than `njit`, though better than the ad-hoc dot product.


In [8]:
%timeit -n 50 _ = matmul(m1, m2)


723 µs ± 91.8 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)
